# `langchain`基础

`langchain`基础包括：

- 提示词模板
- 输出解析器
- 链式调用



## 提示词模板

提示词角色有三种：

- `system`: 系统角色
- `human`: 用户角色，在OpenAI里面是 `user`
- `assistant`: AI角色


In [1]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个省钱小能手，尽最大努力帮用户省钱。'),
        ('human', '我想买一个{product}')
    ]
)
result = prompt_template.invoke({'product': 'MacBook Pro'})
result

ChatPromptValue(messages=[SystemMessage(content='你是一个省钱小能手，尽最大努力帮用户省钱。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我想买一个MacBook Pro', additional_kwargs={}, response_metadata={})])

通过`MessagesPlaceholder`占位符，可以将历史消息或多条消息放入上下文中。

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '你是一个总结小助手'),
    MessagesPlaceholder('history_messages'),
    #  上述MessagesPlaceholder等同于：('placeholder', 'history_messages')
    ('human', '{message}')
])

history_message = [HumanMessage('我的名字叫李明'), HumanMessage('我今年29岁'), HumanMessage('我正在学习AI')]
prompt_template.invoke({'history_messages': history_message, 'message': '请总结一下我的信息'})

ChatPromptValue(messages=[SystemMessage(content='你是一个总结小助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的名字叫李明', additional_kwargs={}, response_metadata={}), HumanMessage(content='我今年29岁', additional_kwargs={}, response_metadata={}), HumanMessage(content='我正在学习AI', additional_kwargs={}, response_metadata={}), HumanMessage(content='请总结一下我的信息', additional_kwargs={}, response_metadata={})])

## 输出解析器

作用是对输出数据进行格式化处理，输出解析器也可以调用大模型对消息进行进一步处理。

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

# 这里调用的是 ollama 本地模型
model = ChatOpenAI(
    model='deepseek-r1:14b',
    base_url='http://192.168.1.188:11434/v1',
    api_key='your_api_key'
)

messages = [
    SystemMessage(content='你是一个翻译高手'),
    HumanMessage(content='请将中文翻译为英文'),
    HumanMessage(content='中午我想吃面条'),
]

# 字符串格式化
parser = StrOutputParser()
result = model.invoke(messages)
print('AI:', result)
response = parser.invoke(result)
print('格式化：', response)

AI: content='<think>\n\n</think>\n\nI want to eat noodles at noon.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 18, 'total_tokens': 31, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-r1:14b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None} id='run-a1359169-a5be-4f71-8f28-793493066ea6-0' usage_metadata={'input_tokens': 18, 'output_tokens': 13, 'total_tokens': 31, 'input_token_details': {}, 'output_token_details': {}}
格式化： <think>

</think>

I want to eat noodles at noon.


## 链式调用

类似`Linux`的管道操作，将多个执行器串联一起输出最终结果。
上述的代码使用链式调用可以写成如下：

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# 这里调用的是 ollama 本地模型
model = ChatOpenAI(
    model='deepseek-r1:14b',
    base_url='http://192.168.1.188:11434/v1',
    api_key='your_api_key'
)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '你是一个总结小助手。'),
    MessagesPlaceholder(variable_name='history_messages'),
    ('human', '{input_message}')
])

history_message = [
    ('human','我的名字叫李明'),
    ('human','我今年29岁'),
    ('human','我正在学习AI'),
    ('human','我喜欢摄影')
]
# 链式调用等同于：prompt_template.invoke && model.invoke && StrOutputParser().invoke
chain =  prompt_template | model | StrOutputParser()
result = chain.invoke(input={'input_message': '请总结一下我的信息', 'history_messages': history_message})
print('AI:', result)

AI: <think>
好吧，用户让我总结他的信息。他提供了自己的名字、年龄、正在学习的内容以及兴趣爱好。我觉得我需要把这些信息组织得清晰一些。

首先，他会叫李明，29岁。然后在学习AI，这可能和职业发展有关。喜欢摄影说明他有艺术方面的兴趣。结合起来，这个总结可以突出他的学习领域和他的个人爱好之间的平衡。

或许我可以分成几个点：基本信息、当前学习、兴趣爱好。这样结构更清晰，也便于阅读。同时，用简洁的语言来表达每个部分，避免冗长。

需要注意的是，不使用任何markdown格式，保持自然的口语化中文。还要确保内容全面，涵盖所有提供的信息，但不过于详细。

最后，检查一下是否有遗漏的信息或者有没有更合适的表达方式。确保总结既准确又易于理解。
</think>

你的信息可以总结如下：

- **的名字：李明  
- **年龄**：29岁  
- **正在学习**：AI  
- **兴趣爱好**：摄影  

希望这样的总结对你有帮助！如果有其他需求，请随时告诉我。


下面简单介绍一下通过`SequentialChain`将多个模型串联起来：

- 通过`input_variables`指定第一个链的输入的`key`
- 通过`output_variables`指定每一个链的输出`key`
- 将上一个模型`output key` 作为下一个模型`input key`

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.chains import SequentialChain, LLMChain
from dotenv import load_dotenv


# 加载智谱开放平台模型的相关参数
load_dotenv()
# 这里调用的是 ollama 本地模型
llm1 = ChatOpenAI(
    model='deepseek-r1:14b',
    base_url='http://192.168.1.188:11434/v1',
    api_key='your_api_key'
)
# 调用智谱开放平台模型，在.env中配置 OPENAI_API_KEY 和 OPENAI_BASE_URL
llm2 = ChatOpenAI (model='glm-4-flash')

prompt1 = ChatPromptTemplate.from_messages([
    ('system', '你是一个总结小助手。'),
    ('human','我的名字叫李明'),
    ('human','我今年29岁'),
    ('human','2022年毕业于湖南大学计算机科学与技术专业'),
    ('human','我正在学习AI'),
    ('human','我喜欢摄影'),
    ('human', '{input}')
])


chain1 = LLMChain(llm=llm1, prompt=prompt1, output_key="summary")

prompt2 = ChatPromptTemplate.from_messages([
    ('system', '你是一个简历小助手。'),
    ('human', '{summary}'),
    ('human', '帮我写一个详细的求职简历')
])
chain2 = LLMChain(llm=llm2, prompt=prompt2, output_key="final_answer")

overall_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["input"],  # 初始输入变量
    output_variables=["summary", "final_answer"],  # 最终输出变量
    verbose=True
)
result = overall_chain.invoke('请总结一下我的信息')
print(result["final_answer"])



> Entering new SequentialChain chain...

> Finished chain.
[李明]

[联系信息]
手机：138xxxx5678
电子邮箱：liming@example.com
LinkedIn：linkedin.com/in/liming
居住地：长沙市

[个人简介]
热情、有责任心的计算机科学与技术专业毕业生，拥有扎实的编程基础和良好的逻辑思维能力。对人工智能领域充满兴趣，并持续学习相关技能。擅长团队协作，具备良好的沟通能力和问题解决能力。渴望在技术领域发挥潜力，为团队贡献自己的力量。

[教育背景]
- 湖南大学
  - 计算机科学与技术专业
  - 学士学位
  - 2018年9月 - 2022年6月

[专业技能]
- 熟练掌握Java、Python、C++等编程语言。
- 熟悉Linux操作系统和常用命令。
- 熟练使用MySQL、Oracle等数据库管理系统。
- 熟悉HTML、CSS、JavaScript等前端技术。
- 了解数据结构与算法设计。
- 在人工智能领域，具备机器学习、深度学习的基础知识，熟悉TensorFlow和PyTorch等框架。

[项目经验]
- **智能图像识别系统**（2021年）
  - 参与开发一款基于深度学习的图像识别系统，用于自动识别和分类图像。
  - 负责模型训练和优化，提高了识别准确率。
  - 使用TensorFlow框架实现，参与系统部署和测试。

- **在线教育平台**（2020年）
  - 参与开发一个在线教育平台，负责用户注册、登录模块的开发。
  - 使用Java和Spring框架进行后端开发，实现用户认证和权限管理。
  - 与前端团队协作，确保系统界面友好，用户体验良好。

[实习经历]
- **XX科技有限公司**
  - 软件开发实习生
  - 2019年7月 - 2019年9月
  - 参与公司内部管理系统的开发，负责部分模块的设计与实现。
  - 协助团队进行代码审查，提高代码质量。

[兴趣爱好]
- 摄影：热爱用镜头捕捉生活中的美好瞬间。
- 阅读：广泛阅读科技、历史、文学等各类书籍，不断丰富知识储备。

[证书]
- 计算机技术与软件专业技术资格（水平）考试——软件设计师
- 英语六级证书

[自我评价]
- 学习能力强，对新知识